Анализ  [данных](https://disk.yandex.ru/d/W3dhYgu5EQOifQ) с целью выявления необходимых признаков, на которых в будущем будет построена модель.

Описание данных:
<dl>
<dd>id - ID поездки </dd>
<dd>vendor_id - ID компании, осуществляющей перевозку </dd>
<dd>pickup_datetime - Таймкод начала поездки</dd>
<dd>dropoff_datetime - Таймкод конца поездки </dd>
<dd>passenger_count - Количество пассажиров </dd>
<dd>pickup_longitude - Долгота точки, в которой началась поездка </dd>
<dd>pickup_latitude - Широта точки, в которой началась поездка </dd>
<dd>dropoff_longitude - Долгота точки, в которой закончилась поездка </dd>
<dd>dropoff_latitude - Широта точки, в которой закончилась поездка </dd>
<dd>store_and_fwd_flag - Yes/No: Была ли информация сохранена в памяти транспортного средства из-за потери соединения с сервером </dd>
</dl>

**Целевая переменная - длительность поездки.**

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

taxiDB = pd.read_csv('taxi_dataset.csv')

In [69]:
taxiDB.head(5)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N


In [70]:
taxiDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 111.3+ MB


In [71]:
### таргетную переменнул в  trip_duration
taxiDB['pickup_datetime'] = pd.to_datetime(taxiDB['pickup_datetime'])
taxiDB['dropoff_datetime'] = pd.to_datetime(taxiDB['dropoff_datetime'])
taxiDB['trip_duration'] = (taxiDB['dropoff_datetime']  - taxiDB['pickup_datetime']).dt.total_seconds()

Предсказывая таргет для новых объектов в будущем, мы не будем заранее знать **dropoff_datetime**.

In [72]:
### Удалим колонку из датасета.
taxiDB = taxiDB.drop(['dropoff_datetime'], axis=1)

In [73]:
taxiDB.head(1)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.96463,40.765602,N,455.0



Бинарный признак vendor_id, принимающий значения {1, 2}. Переведем его во множество {0, 1}, так как это просто привычнее.

In [74]:
taxiDB['vendor_id'] = taxiDB['vendor_id'] - 1
taxiDB.head(3)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,1,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,N,455.0
1,id2377394,0,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,N,663.0
2,id3858529,1,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,N,2124.0


Бинарный признак закодируем его тоже во множество {0, 1}.

In [75]:
### Your code is here
taxiDB['store_and_fwd_flag'] = taxiDB['store_and_fwd_flag'].map(lambda x: 1 if x == 'Y' else 0)

In [76]:
###Первые 10 строк получившегося на этом этапе датафрейма в csv файл с разделителем ';'. 
taxiDB[:10].to_csv('task5.csv', sep = ';')

**Каждую долготу в некоторое относительно километровое выражение**

In [77]:
allLat  = list(taxiDB['pickup_latitude']) + list(taxiDB['dropoff_latitude'])

Посчитаем медиану:

In [78]:
medianLat  = sorted(allLat)[int(len(allLat)/2)]

Из каждого значения широты вычтем медианное значение.

Результат переведем в километры.

In [79]:
latMultiplier  = 111.32

taxiDB['pickup_latitude']   = latMultiplier  * (taxiDB['pickup_latitude']   - medianLat)
taxiDB['dropoff_latitude']   = latMultiplier  * (taxiDB['dropoff_latitude']  - medianLat)

In [80]:
taxiDB.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,1,2016-03-14 17:24:55,1,-73.982155,1.516008,-73.964630,1.256121,0,455.0
1,id2377394,0,2016-06-12 00:43:35,1,-73.980415,-1.753813,-73.999481,-2.578912,0,663.0
2,id3858529,1,2016-01-19 11:35:24,1,-73.979027,1.070973,-74.005333,-4.923841,0,2124.0
3,id3504673,1,2016-04-06 19:32:31,1,-74.010040,-3.823568,-74.012268,-5.298809,0,429.0
4,id2181028,1,2016-03-26 13:30:55,1,-73.973053,4.329328,-73.972923,3.139453,0,435.0



**latitude** - на сколько примерно километров севернее или южнее (в зависимости от знака) точка находится относительно средней широты

In [81]:
allLong = list(taxiDB['pickup_longitude']) + list(taxiDB['dropoff_longitude'])

medianLong  = sorted(allLong)[int(len(allLong)/2)]

longMultiplier = np.cos(medianLat*(np.pi/180.0)) * 111.32

**longitude** признаки

In [82]:
taxiDB['pickup_longitude']   = longMultiplier  * (taxiDB['pickup_longitude']   - medianLong)
taxiDB['dropoff_longitude']   = longMultiplier  * (taxiDB['dropoff_longitude']  - medianLong)

Географическое расстояние **distance_km**:

In [83]:
taxiDB['distance_km']= ((taxiDB['dropoff_longitude'] - taxiDB['pickup_longitude'] )**2 + (taxiDB['dropoff_latitude'] - taxiDB['pickup_latitude'] )**2)**0.5

In [84]:
taxiDB.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,distance_km
0,id2875421,1,2016-03-14 17:24:55,1,-0.110015,1.516008,1.367786,1.256121,0,455.0,1.500479
1,id2377394,0,2016-06-12 00:43:35,1,0.036672,-1.753813,-1.571088,-2.578912,0,663.0,1.807119
2,id3858529,1,2016-01-19 11:35:24,1,0.153763,1.070973,-2.064547,-4.923841,0,2124.0,6.392080
3,id3504673,1,2016-04-06 19:32:31,1,-2.461500,-3.823568,-2.649362,-5.298809,0,429.0,1.487155
4,id2181028,1,2016-03-26 13:30:55,1,0.657515,4.329328,0.668452,3.139453,0,435.0,1.189925


Удаление старых признаков.

In [85]:
taxiDB = taxiDB.drop(['pickup_longitude', 'dropoff_longitude',
                      'pickup_latitude', 'dropoff_latitude'], axis=1)

In [86]:
taxiDB.head()

,id,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
0,id2875421,1,2016-03-14 17:24:55,1,0,455.0,1.500479
1,id2377394,0,2016-06-12 00:43:35,1,0,663.0,1.807119
2,id3858529,1,2016-01-19 11:35:24,1,0,2124.0,6.392080
3,id3504673,1,2016-04-06 19:32:31,1,0,429.0,1.487155
4,id2181028,1,2016-03-26 13:30:55,1,0,435.0,1.189925


In [87]:
### Первые 10 значений полученного промежуточного датафрейма в файл в формате csv с сепаратором ;.
taxiDB[:10].to_csv('task6.csv', sep = ';')

Возможные значения **passenger_count**.


In [88]:
taxiDB.groupby(['passenger_count'])['id'].count()

passenger_count
0         60
1    1033540
2     210318
3      59896
4      28404
5      78088
6      48333
7          3
8          1
9          1
Name: id, dtype: int64

**Mean-target encoding**. Замена колонки **passenger_count** колонкой **category_encoded**.

In [89]:
### Your code is here
taxiDB['category_encoded'] = taxiDB['passenger_count'].map(taxiDB.groupby(['passenger_count'])['trip_duration'].mean())

In [90]:
taxiDB = taxiDB.drop(['passenger_count'], axis=1)

In [91]:
###  Cохраните первые 10 значений полученного промежуточного датафрейма в файл в формате csv с сепаратором ';' Отправьте полученный файл в форму ответа задания 7.
taxiDB[:10].to_csv('task7.csv', sep = ';')

Остались две колонки: **id**, **pickup_datetime**

**id** можно использовать как обычный идентификатор нашего объекта.

In [92]:
taxiDB = taxiDB.set_index('id')

In [93]:
taxiDB.head()

,vendor_id,pickup_datetime,store_and_fwd_flag,trip_duration,distance_km,category_encoded
id,,,,,,
id2875421,1,2016-03-14 17:24:55,0,455.0,1.500479,930.399753
id2377394,0,2016-06-12 00:43:35,0,663.0,1.807119,930.399753
id3858529,1,2016-01-19 11:35:24,0,2124.0,6.392080,930.399753
id3504673,1,2016-04-06 19:32:31,0,429.0,1.487155,930.399753
id2181028,1,2016-03-26 13:30:55,0,435.0,1.189925,930.399753


In [94]:
### Gервые 10 000 значений полученного датафрейма в файл в формате csv с сепаратором ';'.
taxiDB[:10000].to_csv('task8.csv', sep = ';')